In [2]:
USE_IBM_RUNTIME = True

if USE_IBM_RUNTIME:
    from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
    service = QiskitRuntimeService()
    estimator = EstimatorV2(service=service, backend="ibmq_qasm_simulator")
else:
    from qiskit.primitives import Estimator
    estimator = Estimator()

from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper

class MockH2Driver:
    def __init__(self):
        # tady nic nepotřebujeme
        pass
    
    def run(self):
        # Hamiltonian for H2 molecule in STO-3G, 0.735 Å
        h2_ferm_op = FermionicOp({
            "+_0 -_0": -1.252477,
            "+_1 -_1": -0.475934,
            "+_0 -_1 +_1 -_0": 0.174349
        }, num_spin_orbitals=4)
        
        # Mock ElectronicStructureProblem-like object
        class MockProblem:
            def __init__(self, op):
                self.second_q_ops = lambda: [op]
                self.num_spatial_orbitals = 2
                self.num_particles = (1, 1)  # 2 electrons: 1 alpha, 1 beta
                self.nuclear_repulsion_energy = 0.713776
        
        return MockProblem(h2_ferm_op)


# Usage
driver = MockH2Driver()
es_problem = driver.run()
ferm_hamiltonian = es_problem.second_q_ops()[0]
print(ferm_hamiltonian)

mapper = JordanWignerMapper()
qubit_op = mapper.map(ferm_hamiltonian)
print(qubit_op)

# Ansatz: UCCSD with HF initial state
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

# Optional: decompose gates for clarity
ansatz = ansatz.decompose().decompose()

# VQE setup
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=SLSQP())
vqe.initial_point = [0.0] * ansatz.num_parameters

# Run VQE
result = vqe.compute_minimum_eigenvalue(qubit_op)

# Extract results
E_elec = result.eigenvalue.real
E_total = E_elec + es_problem.nuclear_repulsion_energy

print(f"Electronic energy: {E_elec:.6f} Ha")
print(f"Nuclear repulsion: {es_problem.nuclear_repulsion_energy:.6f} Ha")
print(f"Total energy:     {E_total:.6f} Ha")

# Draw ansatz
ansatz.draw("mpl")



AccountNotFoundError: 'Unable to find account.'

ImportError: cannot import name 'Aer' from 'qiskit' (/Users/ema/Documents/Python/quantum/Quantum-Examples/qiskit_env/lib/python3.11/site-packages/qiskit/__init__.py)